# Imbalanced classification: credit card fraud detection
### This example looks at the Kaggle Credit Card Fraud Detection dataset to demonstrate how to train a classification model on data with highly imbalanced classes.

In [1]:
import csv
import numpy as np

## Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/


In [4]:
fname = "Downloads/creditcard.csv"

## vectorize the CSV data


In [6]:
all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])
features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)           

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


## Prepare a validation set to find and optimize the best model 

In [7]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


## Analyze class imbalance in the targets

In [8]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


# Normalize the data using training set statistics This improves the accuracy and integrity of our data, while making our database easier to navigate.

In [9]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

# Build a binary classification model

In [10]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 139,777
Trainable params: 139,777
Non-trai

# Train the model with class_weight argument  to adjust such setting

In [11]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
112/112 - 13s - loss: 2.4827e-06 - fn: 41.0000 - fp: 29692.0000 - tn: 197737.0000 - tp: 376.0000 - precision: 0.0125 - recall: 0.9017 - val_loss: 0.2236 - val_fn: 5.0000 - val_fp: 3198.0000 - val_tn: 53688.0000 - val_tp: 70.0000 - val_precision: 0.0214 - val_recall: 0.9333 - 13s/epoch - 117ms/step
Epoch 2/30
112/112 - 4s - loss: 1.5712e-06 - fn: 32.0000 - fp: 7440.0000 - tn: 219989.0000 - tp: 385.0000 - precision: 0.0492 - recall: 0.9233 - val_loss: 0.0998 - val_fn: 7.0000 - val_fp: 1059.0000 - val_tn: 55827.0000 - val_tp: 68.0000 - val_precision: 0.0603 - val_recall: 0.9067 - 4s/epoch - 32ms/step
Epoch 3/30
112/112 - 4s - loss: 1.3212e-06 - fn: 32.0000 - fp: 7050.0000 - tn: 220379.0000 - tp: 385.0000 - precision: 0.0518 - recall: 0.9233 - val_loss: 0.0943 - val_fn: 9.0000 - val_fp: 767.0000 - val_tn: 56119.0000 - val_tp: 66.0000 - val_precision: 0.0792 - val_recall: 0.8800 - 4s/epoch - 37ms/step
Epoch 4/30
112/112 - 4s - loss: 1.0374e-06 - fn: 28.0000 - fp: 6420.0000 - tn: 

Epoch 28/30
112/112 - 4s - loss: 5.5188e-07 - fn: 11.0000 - fp: 5510.0000 - tn: 221919.0000 - tp: 406.0000 - precision: 0.0686 - recall: 0.9736 - val_loss: 0.0388 - val_fn: 10.0000 - val_fp: 785.0000 - val_tn: 56101.0000 - val_tp: 65.0000 - val_precision: 0.0765 - val_recall: 0.8667 - 4s/epoch - 34ms/step
Epoch 29/30
112/112 - 5s - loss: 4.3319e-07 - fn: 4.0000 - fp: 3642.0000 - tn: 223787.0000 - tp: 413.0000 - precision: 0.1018 - recall: 0.9904 - val_loss: 0.0129 - val_fn: 13.0000 - val_fp: 157.0000 - val_tn: 56729.0000 - val_tp: 62.0000 - val_precision: 0.2831 - val_recall: 0.8267 - 5s/epoch - 42ms/step
Epoch 30/30
112/112 - 5s - loss: 2.9011e-07 - fn: 3.0000 - fp: 2004.0000 - tn: 225425.0000 - tp: 414.0000 - precision: 0.1712 - recall: 0.9928 - val_loss: 0.0316 - val_fn: 9.0000 - val_fp: 682.0000 - val_tn: 56204.0000 - val_tp: 66.0000 - val_precision: 0.0882 - val_recall: 0.8800 - 5s/epoch - 41ms/step


# Conclusion
At the end of training, out of 56,961 validation transactions, we are:

Correctly identifying 66 of them as fraudulent
Missing 9 fraudulent transactions
At the cost of incorrectly flagging 441 legitimate transactions